# 🧪 Evaluating Llama Stack Models with Built-in Evaluation Framework

This notebook demonstrates **Model Evaluation** - a critical process for ensuring AI models perform reliably and meet quality standards before production deployment.

**What is Model Evaluation?**
Model evaluation transforms uncertainty into confidence by:
- **Testing** models against known correct answers and expected behaviors
- **Measuring** performance across different types of questions and tasks
- **Identifying** potential issues and failure modes before deployment
- **Comparing** different models or configurations systematically

**Why Evaluation Matters:**
Instead of hoping your model works correctly, evaluation provides objective metrics and insights:
- **Quality Assurance**: Verify models meet performance thresholds
- **Regression Testing**: Ensure changes don't break existing functionality  
- **Model Selection**: Compare different models to choose the best one
- **Continuous Improvement**: Track performance over time and identify areas for enhancement

**Llama Stack's Evaluation Framework:**
Llama Stack provides three core APIs for comprehensive evaluation:
- 📊 **`/datasetio` + `/datasets`** - Managing evaluation datasets
- 🎯 **`/scoring` + `/scoring_functions`** - Running scoring functions  
- 📈 **`/eval` + `/benchmarks`** - Comprehensive evaluation workflows

Let's explore both strict and flexible evaluation methods to build robust quality assurance! 🚀

## 🏗️ Understanding Evaluation Methods

LlamaStack provides **two primary evaluation approaches** that work together to provide comprehensive model testing:

### 1. 🎯 Exact Matching: `subset_of`
This method performs strict string matching:
- **Speed**: Lightning fast - simple substring search
- **Precision**: Exact character-by-character matching 
- **Use Case**: Perfect for factual questions with clear, specific answers
- **Limitation**: Case-sensitive and inflexible to paraphrasing

**Example**: "Paris" ✅ matches "The capital of France is Paris"  
**Problem**: "shakespeare" ❌ fails to match "William Shakespeare" (case difference)

### 2. 🤖 Semantic Evaluation: `llm_as_judge`  
This method uses AI to understand meaning:
- **Flexibility**: Handles paraphrasing, case differences, and context
- **Intelligence**: Understands semantic equivalence and factual consistency
- **Use Case**: Complex answers requiring interpretation and reasoning
- **Cost**: Requires additional LLM inference for each evaluation

**Example**: "shakespeare" ✅ semantically matches "William Shakespeare wrote Romeo and Juliet"

### 3. 📊 Academic Benchmarks: Multiple Choice + Regex
For standardized testing:
- **Structure**: Multiple choice questions (A, B, C, D format)
- **Parsing**: Regex extraction of letter choices from model responses
- **Scoring**: Binary correct/incorrect based on exact letter match
- **Use Case**: Academic knowledge, reasoning, and comprehension testing

**The Best Approach**: Use both methods together for comprehensive evaluation:
- `subset_of` for quick sanity checks and exact factual verification
- `llm_as_judge` for nuanced understanding and complex reasoning
- Academic benchmarks for standardized performance comparison

## 📦 Install Required Packages

Install the evaluation dependencies:

In [ ]:
!pip install -q llama-stack-client==0.2.12

In [ ]:
# Core imports for evaluation functionality
from llama_stack_client import LlamaStackClient
import pprint  # Built-in Python module for pretty printing

# Additional utilities for evaluation
import json
import sys

## 🔗 Connect to LlamaStack

Connect to LlamaStack - the AI engine that orchestrates all evaluation operations. LlamaStack acts as the central hub that coordinates:
- Model inference for generating responses to evaluation questions
- Scoring functions to compare generated vs expected answers
- Dataset management for organizing evaluation data
- Benchmark execution for comprehensive testing workflows

In [ ]:
# === LlamaStack Connection Setup ===
# The base URL points to your LlamaStack server deployment
base_url = "http://llama-stack-instance-service.llama-serve.svc.cluster.local:8321"

# Create the LlamaStack client - this is your main interface for all evaluation operations
client = LlamaStackClient(
    base_url=base_url,
    timeout=600.0  # Extended timeout for evaluation operations
)

print(f"🔗 Connected to LlamaStack server at {base_url}")

# === Model Configuration ===
# Get available models for evaluation
try:
    available_models = [
        model.identifier for model in client.models.list() if model.model_type == "llm"
    ]
    if available_models:
        model_id = available_models[0]  # Use the first available model
        print(f"✅ Found available model: {model_id}")
    else:
        print("❌ No LLM models found. Please ensure your Llama Stack has models deployed.")
        sys.exit(1)
except Exception as e:
    print(f"❌ Error connecting to LlamaStack: {e}")
    sys.exit(1)

print(f"🤖 Using model: {model_id} for evaluation")

## 🎯 Step 1: Basic Evaluation with `subset_of`

Let's start with the simplest evaluation method: **exact string matching**. This is the foundation of evaluation testing.

**How `subset_of` works:**
1. **Exact Search**: Looks for the expected answer as a substring within the generated response
2. **Case Sensitive**: "paris" ≠ "Paris" - capitalization must match exactly  
3. **Binary Scoring**: Returns 1.0 (correct) or 0.0 (incorrect)
4. **Fast Execution**: No additional LLM calls required

**Perfect for:**
- Factual questions with specific, unambiguous answers
- Quick sanity checks during development
- Testing basic model functionality

**Watch out for:**
- Case sensitivity causing false negatives
- Paraphrasing being marked as incorrect
- Overly strict matching of essentially correct answers

In [ ]:
# === Create Evaluation Examples ===
# These examples will test our model's factual knowledge and demonstrate evaluation scoring

handmade_eval_rows = [
    {
        "input_query": "What is the capital of France?",
        "generated_answer": "The capital of France is Paris.",
        "expected_answer": "Paris",
    },
    {
        "input_query": "Who wrote Romeo and Juliet?",
        "generated_answer": "William Shakespeare wrote Romeo and Juliet.",
        "expected_answer": "shakespeare",  # lowercase - this will fail due to case sensitivity!
    },
    {
        "input_query": "What is 2 + 2?",
        "generated_answer": "The answer is 4.",
        "expected_answer": "4",
    }
]

print("📝 Testing subset_of evaluation with handmade examples:")
print("🔍 Notice the case sensitivity issue in example 2...")
pprint.pprint(handmade_eval_rows)

# === Run subset_of Evaluation ===
# This will perform exact substring matching between generated and expected answers
print("\n🧪 Running subset_of scoring...")

try:
    scoring_response = client.scoring.score(
        input_rows=handmade_eval_rows,
        scoring_functions={"basic::subset_of": None}  # Use default subset_of configuration
    )

    print("\n📊 Raw Results:")
    pprint.pprint(scoring_response)

    # === Analyze Results ===
    results = scoring_response.results['basic::subset_of']
    accuracy = results.aggregated_results['accuracy']['accuracy']
    
    print(f"\n📈 Overall Accuracy: {accuracy:.1%}")
    print(f"✅ Correct answers: {results.aggregated_results['accuracy']['num_correct']}")
    print(f"📊 Total questions: {results.aggregated_results['accuracy']['num_total']}")
    
    # Show individual scores
    print("\n🔍 Individual Question Analysis:")
    for i, (row, score_row) in enumerate(zip(handmade_eval_rows, results.score_rows)):
        status = "✅ PASS" if score_row['score'] == 1.0 else "❌ FAIL"
        print(f"{i+1}. {status} - {row['input_query']}")
        print(f"   Expected: '{row['expected_answer']}'")
        print(f"   Generated: '{row['generated_answer']}'")
        print(f"   Score: {score_row['score']}")
        
        # Explain why Shakespeare failed
        if score_row['score'] == 0.0 and "shakespeare" in row['expected_answer'].lower():
            print(f"   💡 Failed because: looking for 'shakespeare' in 'William Shakespeare...' (case mismatch)")
        print()

except Exception as e:
    print(f"❌ Evaluation failed: {e}")
    print("💡 Make sure your LlamaStack server is running and accessible")

## 🤖 Step 2: LLM-as-Judge Evaluation

Now let's use a more sophisticated approach: **semantic evaluation with an LLM judge**. This addresses the case sensitivity and rigidity issues we saw with `subset_of`.

**How `llm_as_judge` works:**
1. **AI-Powered**: Uses another LLM to evaluate the semantic similarity between answers
2. **Context-Aware**: Understands meaning, not just exact character matching
3. **Flexible**: Handles paraphrasing, case differences, and contextual variations
4. **Explainable**: Provides reasoning for each evaluation decision

**Judge Prompt Strategy:**
Our judge will compare factual content while ignoring style and format differences. It evaluates using these categories:
- **(A)** Generated response is a subset of expected (correct but incomplete)
- **(B)** Generated response is a superset of expected (correct with extra detail)  
- **(C)** Responses contain the same factual details (perfect match)
- **(D)** There is factual disagreement (incorrect)
- **(E)** Answers differ but differences don't matter factually (acceptable variation)

**Important Note**: In production, use a different, more capable model as the judge than the one being evaluated. For this tutorial, we're using the same model for simplicity, but this "self-judging" approach is not ideal for production evaluation.

In [ ]:
# === Define Judge Prompt ===
# This prompt guides the LLM judge to make consistent, explainable evaluation decisions
JUDGE_PROMPT = """
Given a QUESTION and GENERATED_RESPONSE and EXPECTED_RESPONSE.

Compare the factual content. Ignore differences in style, grammar, or punctuation.
Answer by selecting one option:
(A) The GENERATED_RESPONSE is a subset of the EXPECTED_RESPONSE and is fully consistent.
(B) The GENERATED_RESPONSE is a superset of the EXPECTED_RESPONSE and is fully consistent.
(C) The GENERATED_RESPONSE contains all the same details as the EXPECTED_RESPONSE.
(D) There is a disagreement between the responses.
(E) The answers differ, but these differences don't matter factually.

Format: "Answer: One of ABCDE, Explanation: "

QUESTION: {input_query}
GENERATED_RESPONSE: {generated_answer}
EXPECTED_RESPONSE: {expected_answer}
"""

# === Use Same Evaluation Examples ===
# We'll evaluate the same examples to compare with subset_of results
print(f"🤖 Testing LLM-as-judge with {model_id}:")
print("🔍 Watch how the judge handles the 'shakespeare' case sensitivity issue...")

# === Run LLM-as-Judge Evaluation ===
try:
    scoring_response = client.scoring.score(
        input_rows=handmade_eval_rows,
        scoring_functions={
            "llm-as-judge::base": {
                "judge_model": model_id,                              # Use available model as judge
                "prompt_template": JUDGE_PROMPT,                      # Our custom evaluation prompt
                "type": "llm_as_judge",                              # Specify the scoring type
                "judge_score_regexes": ["Answer: (A|B|C|D|E)"],       # Extract letter grade from response
            }
        }
    )

    print("\n📊 LLM-as-Judge Results:")
    results = scoring_response.results['llm-as-judge::base']
    
    # Display detailed results for each question
    for i, (eval_row, score_row) in enumerate(zip(handmade_eval_rows, results.score_rows)):
        print(f"\n{i+1}. Question: {eval_row['input_query']}")
        print(f"   Expected: {eval_row['expected_answer']}")
        print(f"   Generated: {eval_row['generated_answer']}")
        print(f"   Judge Score: {score_row['score']}")
        print(f"   Judge Reasoning: {score_row.get('judge_feedback', 'No feedback provided')}")
        
        # Interpret the judge's decision
        if score_row['score'] in ['A', 'B', 'C', 'E']:
            print("   🎯 Judge Decision: ✅ SEMANTICALLY CORRECT")
        else:
            print("   🎯 Judge Decision: ❌ FACTUALLY INCORRECT")

    print(f"\n🧠 Key Insight: Notice how the LLM judge handles 'shakespeare' vs 'William Shakespeare'")
    print(f"   The judge understands these refer to the same person, unlike exact string matching!")

except Exception as e:
    print(f"❌ LLM-as-judge evaluation failed: {e}")
    print("💡 This might be due to model availability or prompt formatting issues")

## 📚 Step 3: Dataset-based Evaluation with SimpleQA

Now let's move beyond handcrafted examples to **real-world datasets**. This tests how models perform on questions they haven't seen before.

**SimpleQA Dataset:**
- **Source**: Research-grade dataset from HuggingFace/Meta
- **Content**: Factual knowledge questions across diverse topics
- **Difficulty**: Tests specific, verifiable facts that require precise knowledge
- **Examples**: "Who received the IEEE Frank Rosenblatt Award in 2010?"

**Why Dataset Evaluation Matters:**
1. **Unbiased Testing**: Questions you didn't create, reducing selection bias
2. **Scale**: Test across hundreds or thousands of examples automatically
3. **Benchmarking**: Compare your model's performance to published baselines
4. **Real-world Readiness**: Evaluate on the types of questions users actually ask

**Evaluation Workflow:**
1. **Register Dataset**: Connect to external data source (HuggingFace)
2. **Sample Questions**: Get a subset for quick testing
3. **Model Inference**: Generate answers to dataset questions
4. **LLM-as-Judge**: Use semantic evaluation for nuanced comparison
5. **Analyze Results**: Identify patterns in successes and failures

In [ ]:
# === Step 3: Dataset-based Evaluation ===

print("📚 Setting up SimpleQA dataset evaluation...")

try:
    # === Register SimpleQA Dataset ===
    # This connects to the HuggingFace dataset of factual knowledge questions
    print("🔗 Registering SimpleQA dataset...")
    client.datasets.register(
        purpose="eval/messages-answer",                                      # Dataset format type
        source={
            "type": "uri",
            "uri": "huggingface://datasets/llamastack/simpleqa?split=train", # HuggingFace dataset URI
        },
        dataset_id="huggingface::simpleqa",                                 # Local identifier
    )
    print("✅ Dataset registered successfully!")

    # === Sample Questions from Dataset ===
    # Get a small sample for quick testing (you can increase limit for more comprehensive testing)
    print("\n📋 Sampling questions from SimpleQA dataset...")
    eval_rows = client.datasets.iterrows(
        dataset_id="huggingface::simpleqa",
        limit=3,  # Start small for demo - increase to 10-50 for real evaluation
    )

    print("\n🔍 Sample questions from SimpleQA:")
    for i, row in enumerate(eval_rows.data):
        print(f"{i+1}. {row['input_query']}")
        print(f"   Expected: {row['expected_answer']}")
        print()

    # === Create Benchmark ===
    # This defines how we'll evaluate the dataset
    print("🎯 Creating benchmark with LLM-as-judge scoring...")
    client.benchmarks.register(
        benchmark_id="meta-reference::simpleqa",
        dataset_id="huggingface::simpleqa",
        scoring_functions=["llm-as-judge::base"],  # Use our semantic evaluation
    )

    # === Evaluate Model on Dataset ===
    print(f"🤖 Evaluating {model_id} on SimpleQA knowledge questions...")
    print("⏳ This may take a moment as we generate answers and judge them...")
    
    response = client.eval.evaluate_rows(
        benchmark_id="meta-reference::simpleqa",
        input_rows=eval_rows.data,
        scoring_functions=["llm-as-judge::base"],
        benchmark_config={
            "eval_candidate": {
                "type": "model",
                "model": model_id,
                "sampling_params": {
                    "strategy": {"type": "greedy"},  # Deterministic generation
                    "max_tokens": 512,               # Allow detailed answers
                },
            },
        },
    )

    # === Analyze Dataset Results ===
    print("\n📊 SimpleQA Evaluation Results:")
    print("=" * 60)
    
    for i, gen in enumerate(response.generations):
        score = response.scores['llm-as-judge::base'].score_rows[i]
        
        print(f"\n{i+1}. Question: {eval_rows.data[i]['input_query']}")
        print(f"   Expected: {eval_rows.data[i]['expected_answer']}")
        print(f"   Generated: {gen['generated_answer']}")
        print(f"   Judge Score: {score['score']}")
        print(f"   Judge Feedback: {score.get('judge_feedback', 'No feedback')}")
        
        # Categorize performance
        if score['score'] in ['A', 'B', 'C', 'E']:
            print("   📈 Result: ✅ CORRECT/ACCEPTABLE")
        else:
            print("   📉 Result: ❌ INCORRECT/INADEQUATE")

    # === Performance Summary ===
    correct_count = sum(1 for score in response.scores['llm-as-judge::base'].score_rows 
                       if score['score'] in ['A', 'B', 'C', 'E'])
    total_count = len(response.scores['llm-as-judge::base'].score_rows)
    accuracy = correct_count / total_count if total_count > 0 else 0

    print(f"\n🎯 Dataset Performance Summary:")
    print(f"   Correct: {correct_count}/{total_count} ({accuracy:.1%})")
    print(f"   Model: {model_id}")
    print(f"   Dataset: SimpleQA (factual knowledge)")

except Exception as e:
    print(f"❌ Dataset evaluation failed: {e}")
    print("💡 This might be due to dataset availability or network connectivity issues")

## 🎓 Step 4: Academic Benchmark (MMLU-style) Evaluation

Let's test on **standardized academic benchmarks** - the gold standard for comparing model capabilities across institutions and research papers.

**MMLU (Massive Multitask Language Understanding):**
- **Format**: Multiple choice questions (A, B, C, D)
- **Subjects**: 57 academic domains (math, history, science, law, etc.)
- **Difficulty**: High school to graduate level knowledge
- **Scoring**: Binary correct/incorrect based on letter choice

**Key Differences from Previous Methods:**
- **Structured Output**: Must extract specific letter choice (A/B/C/D)
- **System Prompts**: Use academic expert personas for better performance
- **Regex Parsing**: Extract final answer from potentially verbose explanations
- **Standardized**: Results comparable to published research

**Why Academic Benchmarks Matter:**
1. **Research Comparability**: Your results can be compared to published papers
2. **Objective Metrics**: Clear pass/fail criteria with no ambiguity
3. **Comprehensive Coverage**: Tests reasoning across multiple domains
4. **Industry Standard**: Used by major AI companies for model evaluation

In [ ]:
# === Step 4: Academic Benchmark (MMLU-style) Evaluation ===

# === Define System Prompt for Academic Excellence ===
# This prompt helps the model perform better on academic questions by setting the right context
SYSTEM_PROMPT_TEMPLATE = """
You are an expert in {subject} whose job is to answer multiple choice questions.

First, reason about the correct answer.

Then write the answer in the following format where X is exactly one of A,B,C,D:

Answer: X

Make sure X is one of A,B,C,D.

If you are uncertain of the correct answer, guess the most likely one.
"""

# === Sample MMLU-style Questions ===
# In a real evaluation, you'd load thousands of questions from the actual MMLU dataset
mmlu_sample_rows = [
    {
        "input_query": "What is the capital of France?\nA) London\nB) Berlin\nC) Paris\nD) Madrid",
        "expected_answer": "C",
        "chat_completion_input": '[{"role": "user", "content": "What is the capital of France?\\nA) London\\nB) Berlin\\nC) Paris\\nD) Madrid"}]'
    },
    {
        "input_query": "Which of the following is a prime number?\nA) 4\nB) 6\nC) 8\nD) 7",
        "expected_answer": "D",
        "chat_completion_input": '[{"role": "user", "content": "Which of the following is a prime number?\\nA) 4\\nB) 6\\nC) 8\\nD) 7"}]'
    },
    {
        "input_query": "Who wrote 'Romeo and Juliet'?\nA) Charles Dickens\nB) William Shakespeare\nC) Mark Twain\nD) Jane Austen",
        "expected_answer": "B",
        "chat_completion_input": '[{"role": "user", "content": "Who wrote \'Romeo and Juliet\'?\\nA) Charles Dickens\\nB) William Shakespeare\\nC) Mark Twain\\nD) Jane Austen"}]'
    }
]

print("🎓 MMLU-style Multiple Choice Evaluation")
print("📚 Testing academic knowledge with structured multiple choice format...")

for i, row in enumerate(mmlu_sample_rows):
    print(f"\n{i+1}. {row['input_query']}")
    print(f"   Correct Answer: {row['expected_answer']}")

# === Create Academic Expert System Message ===
system_message = {
    "role": "system", 
    "content": SYSTEM_PROMPT_TEMPLATE.format(subject="academic subjects"),
}

print(f"\n🧪 Running MMLU-style evaluation with {model_id}...")

try:
    # === Register MMLU-style Benchmark ===
    client.benchmarks.register(
        benchmark_id="meta-reference::mmlu-sample",
        dataset_id="mmlu-sample", 
        scoring_functions=[],  # We'll use regex parser for multiple choice
    )

    # === Evaluate with Regex Parser for Multiple Choice ===
    # This extracts the letter choice (A/B/C/D) from the model's response
    response = client.eval.evaluate_rows(
        benchmark_id="meta-reference::mmlu-sample",
        input_rows=mmlu_sample_rows,
        scoring_functions=["basic::regex_parser_multiple_choice_answer"],
        benchmark_config={
            "eval_candidate": {
                "type": "model",
                "model": model_id,
                "sampling_params": {
                    "strategy": {
                        "type": "top_p",
                        "temperature": 0.1,  # Slightly creative but mostly deterministic
                        "top_p": 0.95,
                    },
                    "max_tokens": 512,
                },
                "system_message": system_message,  # Use our academic expert prompt
            },
        },
    )

    # === Analyze MMLU Results ===
    print("\n📊 MMLU-style Evaluation Results:")
    print("=" * 70)
    
    for i, gen in enumerate(response.generations):
        score = response.scores['basic::regex_parser_multiple_choice_answer'].score_rows[i]
        
        question_short = mmlu_sample_rows[i]['input_query'].split('?')[0] + "?"
        print(f"\n{i+1}. Question: {question_short}")
        print(f"   Expected: {mmlu_sample_rows[i]['expected_answer']}")
        print(f"   Generated Response: {gen['generated_answer']}")
        print(f"   Extracted Answer: {score.get('parsed_answer', 'Failed to parse')}")
        print(f"   Score: {'✅ CORRECT' if score['score'] == 1.0 else '❌ INCORRECT'} ({score['score']})")

    # === Calculate MMLU Accuracy ===
    results = response.scores['basic::regex_parser_multiple_choice_answer']
    if 'accuracy' in results.aggregated_results:
        accuracy = results.aggregated_results['accuracy']['accuracy']
        correct = results.aggregated_results['accuracy']['num_correct']
        total = results.aggregated_results['accuracy']['num_total']
        
        print(f"\n🎯 MMLU-style Performance Summary:")
        print(f"   Accuracy: {accuracy:.1%} ({correct}/{total})")
        print(f"   Model: {model_id}")
        print(f"   Format: Multiple Choice (A/B/C/D)")
        print(f"   Parsing: Regex extraction of answer letters")
        
        # Provide context for the results
        if accuracy >= 0.8:
            print(f"   🌟 Excellent performance! This model handles structured academic questions well.")
        elif accuracy >= 0.6:
            print(f"   📈 Good performance with room for improvement on academic reasoning.")
        else:
            print(f"   📉 Consider fine-tuning or using more capable models for academic benchmarks.")
    else:
        print("   ⚠️  Could not calculate aggregated accuracy metrics")

except Exception as e:
    print(f"❌ MMLU evaluation failed: {e}")
    print("💡 This might be due to parsing issues or model response formatting problems")

## 🎉 Comprehensive Evaluation Complete!

**What you accomplished:**
- **🎯 Basic Evaluation**: Implemented exact string matching with `subset_of` for fast, precise testing
- **🤖 Semantic Evaluation**: Used `llm_as_judge` for flexible, context-aware evaluation with explanations
- **📚 Dataset Testing**: Evaluated on real-world SimpleQA dataset for unbiased performance measurement
- **🎓 Academic Benchmarks**: Tested structured multiple-choice questions with regex parsing for standardized comparison
- **📊 End-to-End Pipeline**: Built complete evaluation workflows from question to performance metrics

**Key Technical Insights:**
- **Exact vs Semantic**: `subset_of` is fast but rigid; `llm_as_judge` is flexible but requires more resources
- **Dataset Diversity**: Real datasets reveal different failure modes than handcrafted examples  
- **Structured Evaluation**: Academic benchmarks provide standardized, comparable performance metrics
- **Error Analysis**: Each method reveals different aspects of model capabilities and limitations

**Evaluation Method Comparison:**
| Method | Speed | Flexibility | Use Case | Cost |
|--------|-------|-------------|----------|------|
| `subset_of` | ⚡ Fast | 🔒 Rigid | Quick sanity checks | 💰 Free |
| `llm_as_judge` | 🐌 Slower | 🤸 Flexible | Nuanced evaluation | 💰💰 LLM calls |
| Multiple Choice + Regex | ⚡ Fast | 📏 Structured | Academic benchmarks | 💰 Model calls only |

**Production Best Practices:**
1. **Multi-Method Approach**: Use all three methods together for comprehensive coverage
2. **Judge Model Selection**: Use stronger models (Llama 3.3 70B, 405B) as judges, not the model being evaluated
3. **Dataset Rotation**: Regularly test on new, unseen datasets to prevent overfitting
4. **Error Analysis**: Investigate failure patterns to guide model improvements
5. **Benchmark Tracking**: Monitor performance trends over time and model versions

**Advanced Evaluation Patterns to Explore:**
- **Multi-turn Conversations**: Evaluate dialogue consistency and context retention
- **Tool Use Evaluation**: Test agent capabilities with external tools and APIs
- **Safety & Alignment**: Evaluate harmful content detection and refusal behaviors
- **Domain-specific Benchmarks**: Create custom evaluation datasets for your specific use case
- **Human Evaluation**: Combine automated metrics with human judgment for complex tasks

**Next Steps:**
- Scale up evaluation with larger datasets (100s-1000s of examples)
- Implement automated evaluation pipelines for continuous testing
- Create custom evaluation datasets for your specific domain or use case
- Set up performance monitoring and alerting for production model deployment

Your evaluation framework is now production-ready - the foundation for deploying AI systems with confidence! 🚀